In [1]:
import sys
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [2]:
!pip freeze > requirements.txt

In [3]:
def initialize_weights(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [4]:
def initialize_bias(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [5]:
def get_siamese_model(input_shape):
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    vgg16 = tf.keras.applications.VGG16(
    input_shape = input_shape,
    include_top=False,
    weights="imagenet")
    vgg16.trainable=False
    
    flatten_layer = tf.keras.layers.Flatten()
    dense_layer = tf.keras.layers.Dense(256,activation='relu')
    dropout_1 = tf.keras.layers.Dropout(.2, input_shape=(2,))
    batch_1 = tf.keras.layers.BatchNormalization()
    dense_layer_2 = tf.keras.layers.Dense(256,activation='relu')
    dropout_2 = tf.keras.layers.Dropout(.2, input_shape=(2,))
    batch_2 = tf.keras.layers.BatchNormalization()
    dense_layer_3 = tf.keras.layers.Dense(256)
                                          
    vgg16_model = tf.keras.Sequential([
      vgg16,
      flatten_layer,
      dense_layer,
      dropout_1,
      batch_1,
      dense_layer_2,
      dropout_2,
      batch_2,
      dense_layer_3
    ])
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = vgg16_model(left_input)
    encoded_r = vgg16_model(right_input)
        
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [6]:
model = get_siamese_model((96, 96, 3))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 256)          16028224    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256)          0           sequential[0][0]             

In [7]:
optimizer = Adam(lr = 0.00001)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [8]:
model.load_weights(os.path.join(r"C:\Users\sriba\Documents\Python Scripts\Siamese\OneShotDocClassification\weights_vgg","weights.14000.h5"))

In [9]:
def read_img(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (96, 96))
    image = image.reshape(1,96,96,3)
    image = image[...,::-1]
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 256)          16028224    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256)          0           sequential[0][0]             

In [14]:
type(img_1)

numpy.ndarray

In [15]:
img_1.shape

(1, 96, 96, 3)

In [18]:
base = r"C:\Users\sriba\Documents\Python Scripts\Siamese\OneShotDocClassification\val_data\new"
train = os.path.join(base, "train")
test = os.path.join(base, "test")

In [19]:
data_dic = {}
for category in os.listdir(train):
    category_path = os.path.join(train, category)
    data = []
    for file in os.listdir(category_path):
        img = read_img(os.path.join(category_path, file))
        data.append(img)
    data_dic[category] = data

In [20]:
import operator
pair = []
for category in os.listdir(test):
    print("Folder", category)
    category_path = os.path.join(test, category)
    print(len(os.listdir(category_path)))
    for file in os.listdir(category_path):
        img = read_img(os.path.join(category_path, file))
        cat_result = {}
        for key in data_dic:
            #print(key)
            predict_result = []
            train_data = data_dic[key]
            for data in train_data:
                result = model.predict([data, img])
                predict_result.append(result[0][0])
            #print(result)
            cat_result[key] = max(predict_result)
            predicted = max(cat_result.items(), key=operator.itemgetter(1))[0]
        print(predicted, cat_result[predicted])
        #print(cat_result)

Folder ADVE
5
ADVE 0.9362271
ADVE 0.9993293
ADVE 0.9927799
ADVE 0.9909651
ADVE 0.9920372
Folder Email
4
Email 0.91695553
Email 0.9547016
Email 0.9512208
Email 0.9713198
Folder Letter
4
letter 0.05429149
letter 0.7504741
ADVE 0.33699843
letter 0.12653182
